In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/07 18:43:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/07 18:43:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [6]:
df_green.createOrReplaceTempView('green')

In [30]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    DATE_TRUNC('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS revenue_zone,
    -- Revenue calculation 
    SUM(total_amount) AS amount,
    count(1) as number_records
FROM green
where lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1,2
;
""")

In [31]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [21]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [22]:
df_yellow.createOrReplaceTempView('yellow')

In [32]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    DATE_TRUNC('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS revenue_zone,
    -- Revenue calculation 
    SUM(total_amount) AS amount,
    count(1) as number_records
FROM yellow
where tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 1,2
;
""")

In [33]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [42]:
df_yellow_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [43]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [44]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'revenue_zone'], how='outer')

In [46]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [47]:
df_join = spark.read.parquet('data/report/revenue/total')

In [49]:
df_zones = spark.read.parquet('zones/')

In [50]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [52]:
df_result = df_join.join(df_zones, df_join.revenue_zone == df_zones.LocationID)

In [60]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')